# Feature Ranking by Mutual Information

This notebook analyzes all computed features using mutual information with proper statistical significance testing and uniqueness bias mitigation.

## Overview
- **Parallel MI computation** for efficiency with large feature sets
- **Statistical significance testing** against randomized baselines
- **Uniqueness bias mitigation** using sample weights from overlap analysis
- **Comprehensive analysis** of feature effectiveness for triple barrier targets


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import logging
from typing import List, Tuple, Dict, Optional
import warnings
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scientific computing
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import KBinsDiscretizer
from scipy import stats
from statsmodels.stats.multitest import multipletests

# Parallel processing
from joblib import Parallel, delayed
import multiprocessing

# Configure display and logging
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"Available CPU cores: {multiprocessing.cpu_count()}")
print(f"Notebook ready for mutual information analysis")

Available CPU cores: 64
Notebook ready for mutual information analysis


## 1. Data Loading & Preparation

In [3]:
# Configuration
DATA_DIR = Path("../artifacts")
FEATURES_FILE = "features_complete.parquet"  # or features_daily.parquet
TARGETS_FILE = "targets_triple_barrier.parquet"

# Load data files
print("Loading features and targets...")
features_df = pd.read_parquet(DATA_DIR / FEATURES_FILE)
targets_df = pd.read_parquet(DATA_DIR / TARGETS_FILE)

print(f"Features shape: {features_df.shape}")
print(f"Targets shape: {targets_df.shape}")
print(f"\nFeature columns: {len(features_df.columns)}")
print(f"Target columns: {list(targets_df.columns)}")

Loading features and targets...
Features shape: (3762490, 436)
Targets shape: (1180922, 14)

Feature columns: 436
Target columns: ['symbol', 't0', 't_hit', 'hit', 'entry_px', 'top', 'bot', 'h_used', 'price_hit', 'ret_from_entry', 'n_overlapping_trajs', 'weight_overlap', 'weight_class_balance', 'weight_final']


In [4]:
# Data alignment and preparation
def prepare_data_for_analysis(features_df: pd.DataFrame, targets_df: pd.DataFrame) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    """
    Prepare and align features with targets for mutual information analysis.
    
    Returns:
        Tuple of (aligned_features, targets, sample_weights)
    """
    print("Preparing data for mutual information analysis...")
    
    # Ensure date columns are datetime
    if 'date' in features_df.columns:
        features_df['date'] = pd.to_datetime(features_df['date'])
    if 't0' in targets_df.columns:
        targets_df['t0'] = pd.to_datetime(targets_df['t0'])
    
    # Align features with targets (inner join on symbol + date)
    if 'date' in features_df.columns and 't0' in targets_df.columns:
        # Standard alignment: features.date = targets.t0
        aligned_df = features_df.merge(
            targets_df[['symbol', 't0', 'hit', 'weight_final', 'n_overlapping_trajs']],
            left_on=['symbol', 'date'], 
            right_on=['symbol', 't0'],
            how='inner'
        )
    else:
        # Fallback: assume data is already aligned by index
        print("Warning: Using index-based alignment - ensure data is properly aligned")
        aligned_df = pd.concat([features_df, targets_df[['hit', 'weight_final']]], axis=1).dropna()
    
    print(f"Aligned data shape: {aligned_df.shape}")
    
    # Identify feature columns (exclude metadata and target columns)
    exclude_cols = {'symbol', 'date', 't0', 'hit', 'weight_final', 'weight_overlap', 
                    'weight_class_balance', 'n_overlapping_trajs'}
    feature_cols = [col for col in aligned_df.columns if col not in exclude_cols]
    
    print(f"Feature columns identified: {len(feature_cols)}")
    
    # Extract features, targets, and weights
    X = aligned_df[feature_cols].copy()
    y = aligned_df['hit'].values
    
    # Use final combined weights if available, otherwise uniform weights
    if 'weight_final' in aligned_df.columns:
        sample_weights = aligned_df['weight_final'].values
        print(f"Using combined sample weights: min={sample_weights.min():.3f}, max={sample_weights.max():.3f}")
    else:
        sample_weights = np.ones(len(X))
        print("Warning: No sample weights found - using uniform weights")
    
    # Data quality checks
    print(f"\nData Quality Summary:")
    print(f"  Total samples: {len(X):,}")
    print(f"  Target distribution: {dict(zip(*np.unique(y, return_counts=True)))}")
    print(f"  Features with >50% missing: {(X.isnull().mean() > 0.5).sum()}")
    print(f"  Infinite values: {np.isinf(X.select_dtypes(include=[np.number])).sum().sum()}")
    
    return X, y, sample_weights, aligned_df

# Prepare data
X, y, sample_weights, aligned_df = prepare_data_for_analysis(features_df, targets_df)

Preparing data for mutual information analysis...
Aligned data shape: (1180922, 440)
Feature columns identified: 434
Using combined sample weights: min=0.217, max=6.479

Data Quality Summary:
  Total samples: 1,180,922
  Target distribution: {-1: 653665, 0: 190951, 1: 336306}
  Features with >50% missing: 15
  Infinite values: 14


## 2. Parallel Mutual Information Computation

In [5]:
class ParallelMutualInformation:
    """
    Efficient parallel computation of mutual information with significance testing.
    """
    
    def __init__(self, n_jobs=-1, batch_size=50, random_state=42):
        self.n_jobs = n_jobs
        self.batch_size = batch_size
        self.random_state = random_state
        self.discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
        
    def preprocess_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocess features for mutual information computation.
        """
        print("Preprocessing features...")
        X_processed = X.copy()
        
        # Handle missing values
        X_processed = X_processed.fillna(X_processed.median())
        
        # Handle infinite values
        X_processed = X_processed.replace([np.inf, -np.inf], np.nan)
        X_processed = X_processed.fillna(X_processed.median())
        
        # Clip extreme outliers (beyond 5 standard deviations)
        numeric_cols = X_processed.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            mean_val = X_processed[col].mean()
            std_val = X_processed[col].std()
            if std_val > 0:
                lower_bound = mean_val - 5 * std_val
                upper_bound = mean_val + 5 * std_val
                X_processed[col] = np.clip(X_processed[col], lower_bound, upper_bound)
        
        print(f"Preprocessed {len(X_processed.columns)} features")
        return X_processed
    
    def compute_mi_batch(self, feature_batch: pd.DataFrame, y: np.ndarray, 
                        sample_weight: np.ndarray, batch_idx: int) -> Dict:
        """
        Compute mutual information for a batch of features.
        """
        try:
            # Compute MI for this batch
            mi_scores = mutual_info_classif(
                feature_batch.values, y, 
                discrete_features=False,
                sample_weight=sample_weight,
                random_state=self.random_state + batch_idx
            )
            
            return {
                'batch_idx': batch_idx,
                'feature_names': list(feature_batch.columns),
                'mi_scores': mi_scores,
                'n_features': len(feature_batch.columns)
            }
            
        except Exception as e:
            print(f"Error in batch {batch_idx}: {e}")
            return {
                'batch_idx': batch_idx,
                'feature_names': list(feature_batch.columns),
                'mi_scores': np.zeros(len(feature_batch.columns)),
                'n_features': len(feature_batch.columns),
                'error': str(e)
            }
    
    def compute_mutual_information(self, X: pd.DataFrame, y: np.ndarray, 
                                 sample_weight: np.ndarray) -> pd.DataFrame:
        """
        Compute mutual information scores for all features in parallel.
        """
        print(f"Computing mutual information for {len(X.columns)} features...")
        
        # Preprocess features
        X_processed = self.preprocess_features(X)
        
        # Create feature batches
        feature_cols = list(X_processed.columns)
        feature_batches = [feature_cols[i:i+self.batch_size] 
                          for i in range(0, len(feature_cols), self.batch_size)]
        
        print(f"Processing {len(feature_batches)} batches of ~{self.batch_size} features each")
        
        # Parallel computation
        with tqdm(total=len(feature_batches), desc="Computing MI") as pbar:
            def update_progress(result):
                pbar.update(1)
                return result
            
            results = Parallel(n_jobs=self.n_jobs, verbose=0)(
                delayed(self.compute_mi_batch)(
                    X_processed[batch_features], y, sample_weight, i
                )
                for i, batch_features in enumerate(feature_batches)
            )
            
            for _ in results:
                pbar.update(1)
        
        # Combine results
        all_features = []
        all_scores = []
        
        for result in sorted(results, key=lambda x: x['batch_idx']):
            all_features.extend(result['feature_names'])
            all_scores.extend(result['mi_scores'])
            
            if 'error' in result:
                print(f"Warning: Batch {result['batch_idx']} had errors")
        
        # Create results DataFrame
        mi_results = pd.DataFrame({
            'feature': all_features,
            'mi_score': all_scores
        }).sort_values('mi_score', ascending=False).reset_index(drop=True)
        
        print(f"Computed MI for {len(mi_results)} features")
        print(f"MI score range: [{mi_results['mi_score'].min():.6f}, {mi_results['mi_score'].max():.6f}]")
        
        return mi_results

# Initialize MI computer
mi_computer = ParallelMutualInformation(n_jobs=-1, batch_size=50)

# Compute mutual information scores
mi_results = mi_computer.compute_mutual_information(X, y, sample_weights)

Computing mutual information for 434 features...
Preprocessing features...
Preprocessed 434 features
Processing 9 batches of ~50 features each


Computing MI:   0%|          | 0/9 [00:00<?, ?it/s]

Computed MI for 434 features
MI score range: [0.000000, 0.000000]


## 3. Statistical Significance Testing

In [ ]:
class SignificanceTester:
    """
    Test statistical significance of MI scores against randomized baselines.
    """
    
    def __init__(self, n_permutations=1000, n_jobs=-1, random_state=42):
        self.n_permutations = n_permutations
        self.n_jobs = n_jobs
        self.random_state = random_state
        
    def generate_random_baseline(self, X: pd.DataFrame, y: np.ndarray, 
                                sample_weight: np.ndarray, perm_idx: int) -> np.ndarray:
        """
        Generate randomized baseline MI scores for one permutation.
        """
        try:
            # Create random permutation of targets while preserving sample weights
            np.random.seed(self.random_state + perm_idx)
            y_permuted = np.random.permutation(y)
            
            # Compute MI with permuted targets
            mi_scores = mutual_info_classif(
                X.values, y_permuted,
                discrete_features=False,
                sample_weight=sample_weight,
                random_state=self.random_state + perm_idx
            )
            
            return mi_scores
            
        except Exception as e:
            print(f"Error in permutation {perm_idx}: {e}")
            return np.zeros(X.shape[1])
    
    def compute_significance(self, X: pd.DataFrame, y: np.ndarray, 
                           sample_weight: np.ndarray, observed_mi: pd.DataFrame) -> pd.DataFrame:
        """
        Compute statistical significance of observed MI scores.
        """
        print(f"Computing statistical significance with {self.n_permutations} permutations...")
        
        # Preprocess features (same as for observed MI)
        X_processed = mi_computer.preprocess_features(X)
        
        # Generate random baselines in parallel
        print("Generating randomized baselines...")
        with tqdm(total=self.n_permutations, desc="Random baselines") as pbar:
            random_scores = Parallel(n_jobs=self.n_jobs, verbose=0)(
                delayed(self.generate_random_baseline)(
                    X_processed, y, sample_weight, i
                )
                for i in range(self.n_permutations)
            )
            
            for _ in random_scores:
                pbar.update(1)
        
        # Convert to array: [n_permutations, n_features]
        random_scores_array = np.array(random_scores)
        
        print(f"Generated {random_scores_array.shape[0]} random baselines for {random_scores_array.shape[1]} features")
        
        # Compute p-values and effect sizes
        results = []
        feature_names = list(X_processed.columns)
        
        for i, feature in enumerate(feature_names):
            observed_score = observed_mi[observed_mi['feature'] == feature]['mi_score'].iloc[0]
            random_feature_scores = random_scores_array[:, i]
            
            # P-value: proportion of random scores >= observed score
            p_value = (np.sum(random_feature_scores >= observed_score) + 1) / (self.n_permutations + 1)
            
            # Effect size: standardized difference from random baseline
            random_mean = np.mean(random_feature_scores)
            random_std = np.std(random_feature_scores)
            effect_size = (observed_score - random_mean) / (random_std + 1e-8)
            
            results.append({
                'feature': feature,
                'mi_score': observed_score,
                'random_mean': random_mean,
                'random_std': random_std,
                'p_value': p_value,
                'effect_size': effect_size
            })
        
        # Create results DataFrame
        significance_df = pd.DataFrame(results)
        
        # Multiple testing correction
        rejected, p_adjusted, _, _ = multipletests(
            significance_df['p_value'], 
            method='fdr_bh', 
            alpha=0.05
        )
        
        significance_df['p_adjusted'] = p_adjusted
        significance_df['significant'] = rejected
        
        # Add significance categories
        significance_df['significance_level'] = pd.cut(
            significance_df['p_adjusted'],
            bins=[0, 0.001, 0.01, 0.05, 1.0],
            labels=['p<0.001', '0.001≤p<0.01', '0.01≤p<0.05', 'p≥0.05']
        )
        
        # Sort by MI score
        significance_df = significance_df.sort_values('mi_score', ascending=False).reset_index(drop=True)
        
        print(f"Significance testing complete:")
        print(f"  Significant features (FDR<0.05): {significance_df['significant'].sum()}")
        print(f"  Highly significant (p<0.001): {(significance_df['p_adjusted'] < 0.001).sum()}")
        
        return significance_df

# Initialize significance tester
sig_tester = SignificanceTester(n_permutations=1000, n_jobs=-1)

# Compute significance (this may take a while)
print("\n" + "="*50)
print("STARTING SIGNIFICANCE TESTING - THIS MAY TAKE 10-30 MINUTES")
print("="*50)

final_results = sig_tester.compute_significance(X, y, sample_weights, mi_results)

## 4. Results Analysis & Visualization

In [ ]:
# Enhanced feature analysis
def analyze_feature_types(results_df: pd.DataFrame) -> pd.DataFrame:
    """
    Classify and analyze different types of features.
    """
    results_enhanced = results_df.copy()
    
    # Feature type classification
    def classify_feature(feature_name):
        name = feature_name.lower()
        if name.startswith('w_'):
            return 'Weekly'
        elif any(x in name for x in ['rsi', 'macd', 'ema', 'sma', 'ma_']):
            return 'Trend/Momentum'
        elif any(x in name for x in ['vol', 'volatility', 'regime']):
            return 'Volatility'
        elif any(x in name for x in ['volume', 'vol_']):
            return 'Volume'
        elif any(x in name for x in ['hurst', 'fractal']):
            return 'Fractal/Hurst'
        elif any(x in name for x in ['distance', 'zscore', 'breakout']):
            return 'Technical'
        elif any(x in name for x in ['alpha', 'beta', 'relstr', 'xsec']):
            return 'Cross-sectional'
        elif any(x in name for x in ['breadth', 'advance']):
            return 'Market Breadth'
        else:
            return 'Other'
    
    results_enhanced['feature_type'] = results_enhanced['feature'].apply(classify_feature)
    
    # Weekly vs Daily
    results_enhanced['temporal_type'] = results_enhanced['feature'].apply(
        lambda x: 'Weekly' if x.startswith('w_') else 'Daily'
    )
    
    return results_enhanced

# Analyze feature types
final_results = analyze_feature_types(final_results)

# Display top results
print("\n" + "="*80)
print("TOP 20 FEATURES BY MUTUAL INFORMATION")
print("="*80)

top_features = final_results.head(20)[[
    'feature', 'feature_type', 'mi_score', 'p_adjusted', 'effect_size', 'significance_level'
]]

print(top_features.to_string(index=False, float_format='%.6f'))

In [ ]:
# Comprehensive visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Top 20 Features by MI Score',
        'MI Distribution vs Random Baseline', 
        'Feature Type Performance',
        'Weekly vs Daily Features'
    ],
    specs=[
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}]
    ]
)

# 1. Top features bar chart
top_20 = final_results.head(20)
colors = ['red' if p < 0.001 else 'orange' if p < 0.01 else 'yellow' if p < 0.05 else 'gray' 
          for p in top_20['p_adjusted']]

fig.add_trace(
    go.Bar(
        x=top_20['mi_score'],
        y=top_20['feature'],
        orientation='h',
        marker_color=colors,
        name='MI Score'
    ),
    row=1, col=1
)

# 2. MI distribution histogram
fig.add_trace(
    go.Histogram(
        x=final_results['mi_score'],
        nbinsx=50,
        name='Observed MI',
        marker_color='blue',
        opacity=0.7
    ),
    row=1, col=2
)

fig.add_trace(
    go.Histogram(
        x=final_results['random_mean'],
        nbinsx=50,
        name='Random Baseline',
        marker_color='red',
        opacity=0.7
    ),
    row=1, col=2
)

# 3. Feature type performance
type_stats = final_results.groupby('feature_type').agg({
    'mi_score': 'mean',
    'significant': 'sum'
}).reset_index()

fig.add_trace(
    go.Bar(
        x=type_stats['feature_type'],
        y=type_stats['mi_score'],
        name='Avg MI Score',
        marker_color='green'
    ),
    row=2, col=1
)

# 4. Weekly vs Daily comparison
temporal_stats = final_results.groupby('temporal_type').agg({
    'mi_score': ['mean', 'count'],
    'significant': 'sum'
}).reset_index()
temporal_stats.columns = ['temporal_type', 'mean_mi', 'count', 'significant']
temporal_stats['pct_significant'] = temporal_stats['significant'] / temporal_stats['count'] * 100

fig.add_trace(
    go.Bar(
        x=temporal_stats['temporal_type'],
        y=temporal_stats['mean_mi'],
        name='Mean MI Score',
        marker_color='purple'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Mutual Information Feature Analysis Results",
    showlegend=False
)

fig.show()

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"Total features analyzed: {len(final_results):,}")
print(f"Statistically significant features: {final_results['significant'].sum():,} ({final_results['significant'].mean()*100:.1f}%)")
print(f"Highly significant (p<0.001): {(final_results['p_adjusted'] < 0.001).sum():,}")
print(f"Mean MI score: {final_results['mi_score'].mean():.6f}")
print(f"Mean random baseline: {final_results['random_mean'].mean():.6f}")
print(f"Best feature: {final_results.iloc[0]['feature']} (MI={final_results.iloc[0]['mi_score']:.6f})")

print("\nFeature Type Breakdown:")
type_summary = final_results.groupby('feature_type').agg({
    'mi_score': ['count', 'mean'],
    'significant': 'sum'
}).round(4)
print(type_summary)

print("\nTemporal Feature Comparison:")
print(temporal_stats[['temporal_type', 'count', 'mean_mi', 'significant', 'pct_significant']].round(4))

## 5. Uniqueness Bias Analysis

In [ ]:
# Uniqueness bias validation
def validate_uniqueness_bias_correction(aligned_df: pd.DataFrame, final_results: pd.DataFrame):
    """
    Analyze the effectiveness of uniqueness bias correction.
    """
    print("\n" + "="*60)
    print("UNIQUENESS BIAS ANALYSIS")
    print("="*60)
    
    if 'n_overlapping_trajs' not in aligned_df.columns:
        print("Warning: No overlap information available for bias analysis")
        return
    
    # Compute unweighted MI for comparison
    print("Computing unweighted MI scores for comparison...")
    X_processed = mi_computer.preprocess_features(X)
    
    unweighted_mi = mutual_info_classif(
        X_processed.values, y,
        discrete_features=False,
        random_state=42
    )
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame({
        'feature': list(X_processed.columns),
        'weighted_mi': final_results['mi_score'],
        'unweighted_mi': unweighted_mi
    })
    
    # Analyze correlation with overlap counts
    feature_overlap_stats = []
    
    for feature in comparison_df['feature'].head(50):  # Top 50 features
        if feature in aligned_df.columns:
            feature_data = aligned_df[[feature, 'n_overlapping_trajs']].dropna()
            if len(feature_data) > 10:
                correlation = feature_data[feature].corr(feature_data['n_overlapping_trajs'])
                feature_overlap_stats.append({
                    'feature': feature,
                    'overlap_correlation': correlation
                })
    
    overlap_stats_df = pd.DataFrame(feature_overlap_stats)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Weighted vs Unweighted MI scatter
    axes[0,0].scatter(comparison_df['unweighted_mi'], comparison_df['weighted_mi'], alpha=0.6)
    axes[0,0].plot([0, comparison_df[['weighted_mi', 'unweighted_mi']].max().max()], 
                   [0, comparison_df[['weighted_mi', 'unweighted_mi']].max().max()], 'r--')
    axes[0,0].set_xlabel('Unweighted MI')
    axes[0,0].set_ylabel('Weighted MI')
    axes[0,0].set_title('Weighted vs Unweighted MI Scores')
    
    # 2. Difference analysis
    comparison_df['mi_difference'] = comparison_df['weighted_mi'] - comparison_df['unweighted_mi']
    axes[0,1].hist(comparison_df['mi_difference'], bins=50, alpha=0.7)
    axes[0,1].axvline(0, color='red', linestyle='--')
    axes[0,1].set_xlabel('MI Difference (Weighted - Unweighted)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title('Distribution of MI Score Differences')
    
    # 3. Sample weight distribution
    axes[1,0].hist(sample_weights, bins=50, alpha=0.7)
    axes[1,0].set_xlabel('Sample Weight')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Sample Weights')
    
    # 4. Overlap correlation analysis
    if len(overlap_stats_df) > 0:
        axes[1,1].hist(overlap_stats_df['overlap_correlation'].dropna(), bins=20, alpha=0.7)
        axes[1,1].axvline(0, color='red', linestyle='--')
        axes[1,1].set_xlabel('Correlation with Overlap Count')
        axes[1,1].set_ylabel('Frequency')
        axes[1,1].set_title('Feature-Overlap Correlations')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"Sample weight statistics:")
    print(f"  Min: {sample_weights.min():.4f}, Max: {sample_weights.max():.4f}")
    print(f"  Mean: {sample_weights.mean():.4f}, Std: {sample_weights.std():.4f}")
    
    print(f"\nMI Score Comparison:")
    print(f"  Mean weighted MI: {comparison_df['weighted_mi'].mean():.6f}")
    print(f"  Mean unweighted MI: {comparison_df['unweighted_mi'].mean():.6f}")
    print(f"  Correlation: {comparison_df['weighted_mi'].corr(comparison_df['unweighted_mi']):.4f}")
    
    significant_changes = (abs(comparison_df['mi_difference']) > 0.001).sum()
    print(f"  Features with significant changes: {significant_changes} ({significant_changes/len(comparison_df)*100:.1f}%)")
    
    if len(overlap_stats_df) > 0:
        high_overlap_corr = (abs(overlap_stats_df['overlap_correlation']) > 0.3).sum()
        print(f"\nOverlap bias indicators:")
        print(f"  Features with high overlap correlation: {high_overlap_corr}/{len(overlap_stats_df)}")

# Run bias analysis
validate_uniqueness_bias_correction(aligned_df, final_results)

## 6. Export Results

In [ ]:
# Save comprehensive results
output_dir = Path("../artifacts")
output_dir.mkdir(exist_ok=True)

# Main results file
final_results.to_csv(output_dir / "feature_ranking_mutual_information.csv", index=False)
final_results.to_parquet(output_dir / "feature_ranking_mutual_information.parquet", index=False)

# Top features only
top_significant = final_results[final_results['significant']].head(100)
top_significant.to_csv(output_dir / "top_significant_features.csv", index=False)

# Summary statistics
summary_stats = {
    'analysis_date': pd.Timestamp.now().isoformat(),
    'total_features': len(final_results),
    'significant_features': final_results['significant'].sum(),
    'highly_significant': (final_results['p_adjusted'] < 0.001).sum(),
    'mean_mi_score': final_results['mi_score'].mean(),
    'best_feature': final_results.iloc[0]['feature'],
    'best_mi_score': final_results.iloc[0]['mi_score'],
    'sample_size': len(aligned_df),
    'n_permutations': sig_tester.n_permutations
}

import json
with open(output_dir / "mutual_information_analysis_summary.json", 'w') as f:
    json.dump(summary_stats, f, indent=2)

print("\n" + "="*60)
print("ANALYSIS COMPLETE - RESULTS EXPORTED")
print("="*60)
print(f"Comprehensive results: {output_dir / 'feature_ranking_mutual_information.csv'}")
print(f"Top significant features: {output_dir / 'top_significant_features.csv'}")
print(f"Analysis summary: {output_dir / 'mutual_information_analysis_summary.json'}")

print(f"\nKey findings:")
print(f"  🏆 Best feature: {summary_stats['best_feature']} (MI={summary_stats['best_mi_score']:.6f})")
print(f"  ✅ Significant features: {summary_stats['significant_features']:,}/{summary_stats['total_features']:,} ({summary_stats['significant_features']/summary_stats['total_features']*100:.1f}%)")
print(f"  🎯 Highly significant: {summary_stats['highly_significant']:,} (p<0.001)")
print(f"  📊 Mean MI score: {summary_stats['mean_mi_score']:.6f}")